In [20]:
# Importar .xlsx
import pandas as pd
def importar_xlsx(archivo):
    """
    Importa un archivo .xlsx y devuelve un DataFrame de pandas.
    
    :param archivo: Ruta al archivo .xlsx
    :return: DataFrame de pandas con los datos del archivo
    """
    try:
        df = pd.read_excel(archivo)
        return df
    except Exception as e:
        print(f"Error al importar el archivo: {e}")
        return None



In [21]:
fc_folios = importar_xlsx('FC_FOLIOS.xlsx')
rs_linfol = importar_xlsx('RS_LINFOL.xlsx')
rs_prolin = importar_xlsx('RS_PROLIN.xlsx')
# Verificar si los DataFrames se importaron correctamente
if fc_folios is not None:
    print("FC_FOLIOS importado correctamente.")
if rs_linfol is not None:
    print("RS_LINFOL importado correctamente.")
if rs_prolin is not None:
    print("RS_PROLIN importado correctamente.")
# Mostrar los tamaños de los DataFrames
print(f"Tamaño de FC_FOLIOS: {fc_folios.shape}")
print(f"Tamaño de RS_LINFOL: {rs_linfol.shape}")
print(f"Tamaño de RS_PROLIN: {rs_prolin.shape}")

FC_FOLIOS importado correctamente.
RS_LINFOL importado correctamente.
RS_PROLIN importado correctamente.
Tamaño de FC_FOLIOS: (1000, 110)
Tamaño de RS_LINFOL: (1000, 119)
Tamaño de RS_PROLIN: (1000, 61)


# Análisis Crítico

En este documento se presenta una exploración inicial de las tablas `fc_folios`, `rs_linfol` y `rs_prolin`, con el objetivo de identificar y comprender el propósito de las columnas que aparentan estar duplicadas. Esta observación se basa en la detección de nombres de columna que comparten una raíz común y que se diferencian únicamente por un sufijo numérico precedido por un punto, tal como se muestra en los siguientes ejemplos:

- Hotel  
- Hotel.1  
- Hotel.2  
- Moneda  
- Moneda.1

Este patrón sugiere una posible duplicación automática de nombres por parte de la herramienta de procesamiento (posiblemente `pandas`) al momento de cargar o combinar datos provenientes de SAP FIORI.

## Método

Se desarrollará un proceso automatizado para detectar columnas duplicadas basadas en su nombre. La lógica de detección considerará columnas que comparten el mismo prefijo y se diferencian únicamente por un sufijo del tipo `.n`, donde `n` es un número entero.

Para cada conjunto de columnas detectadas como duplicadas, se generará un DataFrame con los datos correspondientes y se exportará a un archivo Excel independiente. Por ejemplo:

- `Hotel.xlsx` contendrá las columnas `Hotel`, `Hotel.1`, `Hotel.2` con sus respectivas filas.
- `Moneda.xlsx` incluirá `Moneda` y `Moneda.1`.
- Si no existen duplicados de una columna, no se generará ningún archivo.

Este método tiene como finalidad documentar visualmente los patrones de duplicación y facilitar su análisis posterior.

## Objetivo Principal

Determinar si es viable eliminar columnas duplicadas en los archivos exportados desde SAP FIORI, sin comprometer la integridad o completitud de la información. Esto permitiría simplificar las tablas, reducir redundancia y facilitar el análisis automatizado de reservas de hoteles y comercializadoras.

## Paso 1: Identificación de Columnas con Sufijos Numéricos

El primer paso consiste en identificar columnas cuyos nombres siguen el patrón `nombre`, `nombre.1`, `nombre.2`, etc. Esta decisión nace de una observación empírica al inspeccionar manualmente los DataFrames generados en Python. Se desconoce si dicho comportamiento es generado por `pandas` al importar archivos con encabezados repetidos o por algún proceso previo en la exportación desde SAP.

Aunque esta hipótesis se basa en la experiencia directa, será verificada y sistematizada mediante el proceso de análisis propuesto.

---

## Aspectos a Profundizar Más Adelante (Sugerencias)

- Comparación de contenido entre las columnas duplicadas: ¿son idénticas, complementarias o contienen datos diferentes?
- Indagar si el problema ocurre durante la exportación desde SAP o al cargar el archivo con `pandas`.
- Documentar los hallazgos con capturas de pantalla o gráficos que ayuden a visualizar mejor las columnas repetidas y su contenido.
- Evaluar el impacto de eliminar columnas en el análisis posterior: ¿se pierde información útil?



# Exportación de columnas duplicadas

In [22]:
import os
import pandas as pd

def exportar_columnas_duplicadas(df, nombre_tabla, carpeta_salida="ColumnasDuplicadas"):
    """
    Detecta grupos de columnas duplicadas (por nombre base) y exporta un archivo .xlsx por grupo.
    
    :param df: DataFrame a analizar
    :param nombre_tabla: Nombre de la tabla (para nombrar archivos)
    :param carpeta_salida: Carpeta donde se guardarán los archivos .xlsx
    """

    # Crear carpeta si no existe
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)

    # Contenedor de columnas por nombre base
    columnas_por_base = {}

    for col in df.columns:
        # Separamos por punto: "Hotel.1" → ["Hotel", "1"]
        base = col.split('.')[0]
        if base not in columnas_por_base:
            columnas_por_base[base] = [col]
        else:
            columnas_por_base[base].append(col)

    # Exportar solo los grupos con más de una columna (es decir, duplicados)
    for base, cols in columnas_por_base.items():
        if len(cols) > 1:
            df_grupo = df[cols]
            nombre_archivo = f"{nombre_tabla}_{base}.xlsx"
            ruta = os.path.join(carpeta_salida, nombre_archivo)
            df_grupo.to_excel(ruta, index=False)
            print(f"Exportado: {ruta}")

In [23]:
# Ejecutamos la función para cada tabla
exportar_columnas_duplicadas(fc_folios, "FC_FOLIOS")
exportar_columnas_duplicadas(rs_linfol, "RS_LINFOL")
exportar_columnas_duplicadas(rs_prolin, "RS_PROLIN")


Exportado: ColumnasDuplicadas/FC_FOLIOS_Indicador impuestos.xlsx
Exportado: ColumnasDuplicadas/FC_FOLIOS_Número de Documento Comercial.xlsx
Exportado: ColumnasDuplicadas/FC_FOLIOS_Impuestos incluidos.xlsx
Exportado: ColumnasDuplicadas/FC_FOLIOS_Se ha cambiado la descripción.xlsx
Exportado: ColumnasDuplicadas/FC_FOLIOS_Clasificación fiscal.xlsx
Exportado: ColumnasDuplicadas/RS_LINFOL_Posición.xlsx
Exportado: ColumnasDuplicadas/RS_LINFOL_Base imponible.xlsx
Exportado: ColumnasDuplicadas/RS_LINFOL_Indicador impuestos.xlsx
Exportado: ColumnasDuplicadas/RS_LINFOL_Cambio para la determinación de precios.xlsx
Exportado: ColumnasDuplicadas/RS_LINFOL_Número de Documento Comercial.xlsx
Exportado: ColumnasDuplicadas/RS_LINFOL_Se ha cambiado la descripción.xlsx
Exportado: ColumnasDuplicadas/RS_LINFOL_Importe redondeo.xlsx
Exportado: ColumnasDuplicadas/RS_LINFOL_Hu.xlsx
Exportado: ColumnasDuplicadas/RS_LINFOL_Casilla de selección.xlsx
Exportado: ColumnasDuplicadas/RS_PROLIN_Identificador de concept

### **Se exportaron**

# Comparar valor de las columnas

In [24]:
import os
import pandas as pd

def exportar_columnas_duplicadas_con_verificacion(df, nombre_tabla, carpeta_salida="ColumnasDuplicadas_conVerificacion"):
    """
    Detecta grupos de columnas duplicadas (por nombre base), agrega columna 'Coinciden'
    que indica si los valores en la fila son iguales en todas las columnas del grupo,
    y exporta un archivo .xlsx por grupo.

    :param df: DataFrame a analizar
    :param nombre_tabla: Nombre de la tabla (para nombrar archivos)
    :param carpeta_salida: Carpeta donde se guardarán los archivos .xlsx
    """

    # Crear carpeta si no existe
    if not os.path.exists(carpeta_salida):
        os.makedirs(carpeta_salida)

    # Contenedor de columnas por nombre base
    columnas_por_base = {}

    for col in df.columns:
        # Separamos por punto: "Hotel.1" → ["Hotel", "1"]
        base = col.split('.')[0]
        if base not in columnas_por_base:
            columnas_por_base[base] = [col]
        else:
            columnas_por_base[base].append(col)

    # Exportar solo los grupos con más de una columna (es decir, duplicados)
    for base, cols in columnas_por_base.items():
        if len(cols) > 1:
            df_grupo = df[cols].copy()

            # Función para comparar los valores de cada fila
            def comparar_fila(fila):
                # Todos NaN → True
                if fila.isna().all():
                    return True
                valores = fila.unique()
                return len(valores) == 1

            # Añadir columna "Coinciden"
            df_grupo["Coinciden"] = df_grupo.apply(comparar_fila, axis=1)

            # Guardar archivo
            nombre_archivo = f"{nombre_tabla}_{base}.xlsx"
            ruta = os.path.join(carpeta_salida, nombre_archivo)
            df_grupo.to_excel(ruta, index=False)
            print(f"Exportado: {ruta}")


In [25]:
# Ejecutamos la función para cada tabla con verificación
exportar_columnas_duplicadas_con_verificacion(fc_folios, "FC_FOLIOS")
exportar_columnas_duplicadas_con_verificacion(rs_linfol, "RS_LINFOL")
exportar_columnas_duplicadas_con_verificacion(rs_prolin, "RS_PROLIN")

Exportado: ColumnasDuplicadas_conVerificacion/FC_FOLIOS_Indicador impuestos.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/FC_FOLIOS_Número de Documento Comercial.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/FC_FOLIOS_Impuestos incluidos.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/FC_FOLIOS_Se ha cambiado la descripción.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/FC_FOLIOS_Clasificación fiscal.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/RS_LINFOL_Posición.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/RS_LINFOL_Base imponible.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/RS_LINFOL_Indicador impuestos.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/RS_LINFOL_Cambio para la determinación de precios.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/RS_LINFOL_Número de Documento Comercial.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/RS_LINFOL_Se ha cambiado la descripción.xlsx
Exportado: ColumnasDuplicadas_conVerificacion/RS_LINFOL_Importe 

### **Se exportaron**

### Cuántas filas conciden dentro del grupo

In [29]:
import os
import pandas as pd

# Ruta de la carpeta con los archivos verificados
carpeta = "ColumnasDuplicadas_conVerificacion"

# Recorrer todos los archivos .xlsx en la carpeta y calcular el porcentaje de coincidencia
resultados = []
for archivo in os.listdir(carpeta):
    if archivo.endswith(".xlsx"):
        ruta = os.path.join(carpeta, archivo)
        df_temp = pd.read_excel(ruta)
        if "Coinciden" in df_temp.columns:
            total = len(df_temp)
            trues = df_temp["Coinciden"].sum()
            porcentaje = (trues / total) * 100 if total > 0 else 0
            resultados.append({
                "archivo": archivo,
                "total_filas": total,
                "Coinciden_True": trues,
                "Coinciden_False": total - trues,
                "porcentaje_Coincidencia": porcentaje
            })

# Mostrar resultados
for r in resultados:
    print(f"{r['archivo']}: {r['porcentaje_Coincidencia']:.2f}% ({r['Coinciden_True']} True / {r['total_filas']} filas)")

# Guardar resultados en un DataFrame y exportar a Excel
df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("Resultados_Coincidencia.xlsx", index=False)

FC_FOLIOS_Número de Documento Comercial.xlsx: 97.70% (977 True / 1000 filas)
RS_LINFOL_Indicador impuestos.xlsx: 0.00% (0 True / 1000 filas)
RS_LINFOL_Posición.xlsx: 0.00% (0 True / 1000 filas)
RS_PROLIN_Identificador de concepto.xlsx: 0.00% (0 True / 1000 filas)
RS_LINFOL_Base imponible.xlsx: 100.00% (1000 True / 1000 filas)
FC_FOLIOS_Clasificación fiscal.xlsx: 100.00% (1000 True / 1000 filas)
RS_PROLIN_Marca de descuento.xlsx: 100.00% (1000 True / 1000 filas)
RS_LINFOL_Casilla de selección.xlsx: 100.00% (1000 True / 1000 filas)
RS_LINFOL_Número de Documento Comercial.xlsx: 100.00% (1000 True / 1000 filas)
RS_LINFOL_Importe redondeo.xlsx: 100.00% (1000 True / 1000 filas)
FC_FOLIOS_Impuestos incluidos.xlsx: 70.90% (709 True / 1000 filas)
FC_FOLIOS_Indicador impuestos.xlsx: 0.00% (0 True / 1000 filas)
RS_PROLIN_Número de Documento Comercial.xlsx: 100.00% (1000 True / 1000 filas)
FC_FOLIOS_Se ha cambiado la descripción.xlsx: 99.50% (995 True / 1000 filas)
RS_PROLIN_Precio 6 decimales pre

### **10 de 19 tablas son 100% coincidentes**
### **6 de 19 tablas son 0% coincidentes** -> Y se debe a que una columna es nulas

### DataFrame resumen estructurado

In [27]:
import pandas as pd

def generar_resumen_columnas_duplicadas(lista_tablas, nombres_tablas):
    """
    Genera un DataFrame resumen con detalles de las columnas duplicadas.

    :param lista_tablas: Lista de DataFrames a analizar.
    :param nombres_tablas: Lista de nombres correspondientes a cada DataFrame.
    :return: DataFrame resumen consolidado.
    """
    resumen = []  # Aquí guardaremos los diccionarios con la info

    for df, nombre_tabla in zip(lista_tablas, nombres_tablas):
        columnas_por_base = {}
        for col in df.columns:
            base = col.split('.')[0]  # Extraer el nombre base, ej. "Hotel.1" → "Hotel"
            columnas_por_base.setdefault(base, []).append(col)

        for base, cols in columnas_por_base.items():
            if len(cols) > 1:  # Solo nos interesan grupos duplicados
                for col in cols:
                    tipo = df[col].dtype
                    valores = df[col].unique().tolist()
                    resumen.append({
                        "nombreTabla": nombre_tabla,
                        "grupo": base,
                        "columna": col,
                        "tipoValor": str(tipo),
                        "valoresÚnicos": valores
                    })

    df_resumen = pd.DataFrame(resumen)
    return df_resumen


In [28]:
# Lista de DataFrames
tablas = [fc_folios, rs_linfol, rs_prolin]
nombres = ["fc_folios", "rs_linfol", "rs_prolin"]

# Generar el resumen
df_resumen = generar_resumen_columnas_duplicadas(tablas, nombres)

# Ver los primeros resultados
print(df_resumen.head())

# (Opcional) Exportar a Excel
df_resumen.to_excel("Resumen_ColumnasDuplicadas.xlsx", index=False)


  nombreTabla                          grupo                          columna  \
0   fc_folios            Indicador impuestos              Indicador impuestos   
1   fc_folios            Indicador impuestos            Indicador impuestos.1   
2   fc_folios  Número de Documento Comercial    Número de Documento Comercial   
3   fc_folios  Número de Documento Comercial  Número de Documento Comercial.1   
4   fc_folios            Impuestos incluidos              Impuestos incluidos   

  tipoValor                                      valoresÚnicos  
0   float64                                              [nan]  
1    object                                       [DB, NS, DC]  
2    object  [nan, HDS5A06441, HDS5A06419, HDR5A13603, HDR5...  
3    object  [nan, HDS5A06441, HDS5A06419, HDR5A13603, HDR5...  
4    object                                           [X, nan]  


## Conlusiones

- Comparación de contenido entre las columnas duplicadas: ¿son idénticas, complementarias o contienen datos diferentes?
> Más del **_50%_** de las tablas son **100% Coincidentes**
- Indagar si el problema ocurre durante la exportación desde SAP o al cargar el archivo con `pandas`.
> Ocurre al cargar el archivo con **`Pandas`**
- Documentar los hallazgos con capturas de pantalla o gráficos que ayuden a visualizar mejor las columnas repetidas y su contenido.
> Ya xd
- Evaluar el impacto de eliminar columnas en el análisis posterior: **¿se pierde información útil?**
> En las que hay 100% de Coincidencia: **Negativo** -> **Se procede a eliminación de la columna duplicada**
> 
> En las que hay columna nula: **Negativo** -> **Se procede a eliminación de la columna nula**
>
> En las que hay discrepancia de algún tipo: **Afirmativo** -> **Se mantienen ambas columnas**